# <span style="color:rgb(255, 0, 255)"> Working with APIs
 
<span style="color:rgb(255, 0, 255)"> Ainara Guerra

**Lab instructions**

Following the class example, create a function that returns the price, names of origin and arrival airports and the name of the company. Do it for all the flights between two dates that cost the same.

In [ ]:
# I choose tripadvisor for this lab.
# This is how I did it: 
    # 1 - I search for an API that gave me price and airline (not every flights API does it)
    # 2 - I tried few of them and then stayed with Tripadvisor.
    # 3 - I tried to understand the JSON file because its pretty new to me to work with it. 
    # 4 - I almost cry because the way I thought It could worked didn't work at the end. 
    # 5 - I ask Google and Stack over flow. 
    # 6 - Did some other tries but the flights appeared repeated, don0t know why. 
    # 7 - I debugged with Chat GPT. Now it looks like this: 
    # Plus - I was very frustated with this lab ("drama entered the chat"). 


In [4]:
import requests
import pandas as pd
import json

In [3]:

url = "https://tripadvisor16.p.rapidapi.com/api/v1/flights/searchFlights"

querystring = {"sourceAirportCode":"MAD","destinationAirportCode":"LIS","date":"2023-08-08","itineraryType":"ONE_WAY","sortOrder":"PRICE","numAdults":"1","numSeniors":"0","classOfService":"ECONOMY","pageNumber":"1","currencyCode":"USD"}

headers = {
	"X-RapidAPI-Key": "",
	"X-RapidAPI-Host": "tripadvisor16.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())

{'status': True, 'message': 'Success', 'timestamp': 1688216284185, 'data': {'session': {'searchHash': 'd8c406426649aa9582e97954949f08a5', 'pageLoadUid': '82330469-aed3-4f4b-96d4-643913f56d5b', 'searchId': 'cd660698-00ab-401e-a7d7-63b3861bdf9e.352'}, 'complete': False, 'numOfFilters': 12, 'totalNumResults': 12, 'flights': [{'segments': [{'legs': [{'originStationCode': 'MAD', 'isDifferentOriginStation': False, 'destinationStationCode': 'LIS', 'isDifferentDestinationStation': False, 'departureDateTime': '2023-08-08T23:15:00+02:00', 'arrivalDateTime': '2023-08-08T23:40:00+01:00', 'classOfService': 'ECONOMY', 'marketingCarrierCode': 'EC', 'operatingCarrierCode': 'EC', 'amenities': [], 'flightNumber': 6724, 'seatGuruEquipmentId': 0, 'seatGuruAirlineUrl': '', 'numStops': 0, 'distanceInKM': 514.13513, 'isInternational': True, 'selfTransfer': False, 'operatingCarrier': {'locationId': 0, 'code': 'EC', 'logoUrl': 'https://static.tacdn.com/img2/flights/airlines/logos/100x100/api_default.png', 'dis

In [7]:
r_json = response.json()

In [9]:

def get_flight_info(origin, destination, date1, date2):
    url = "https://tripadvisor16.p.rapidapi.com/api/v1/flights/searchFlights"

    querystring = {
        "sourceAirportCode": origin,
        "destinationAirportCode": destination,
        "date": date1,
        "itineraryType": "ONE_WAY",
        "sortOrder": "PRICE",
        "numAdults": "1",
        "numSeniors": "0",
        "classOfService": "ECONOMY",
        "pageNumber": "1",
        "currencyCode": "USD"
    }

    headers = {
        "X-RapidAPI-Key": "",
        "X-RapidAPI-Host": "tripadvisor16.p.rapidapi.com"
    }

    response1 = requests.get(url, headers=headers, params=querystring)
    data1 = response1.json()

    querystring["date"] = date2

    response2 = requests.get(url, headers=headers, params=querystring)
    data2 = response2.json()

    flights1 = data1['data']['flights']
    flights2 = data2['data']['flights']

    grouped_flights = {}

    for flight in flights1 + flights2:
        price = round(flight['purchaseLinks'][0]['totalPrice'], 2)
        date = flight['segments'][0]['legs'][0]['departureDateTime'].split('T')[0]
        time = flight['segments'][0]['legs'][0]['departureDateTime'].split('T')[1].split('+')[0]
        airline = flight['segments'][0]['legs'][0]['marketingCarrier']['displayName']
        key = f"{price}_{date}"

        if key not in grouped_flights:
            grouped_flights[key] = {
                'price': price,
                'origin': flight['segments'][0]['legs'][0]['originStationCode'],
                'destination': flight['segments'][0]['legs'][0]['destinationStationCode'],
                'flight_options': []
            }

        flight_option = {
            'airline': airline,
            'flight_date': date,
            'flight_time': time
        }

        grouped_flights[key]['flight_options'].append(flight_option)

    filtered_flights = []

    for group in grouped_flights.values():
        if len(group['flight_options']) > 1:
            filtered_flights.append(group)

    return filtered_flights


origin = "MAD"
destination = "LIS"
date1 = "2023-08-08"
date2 = "2023-08-09"

flights = get_flight_info(origin, destination, date1, date2)

if len(flights) == 0:
    print("No flights found for the given criteria.")
else:
    for flight in flights:
        print(f"For this price: {flight['price']}")
        for option in flight['flight_options']:
            print(f"You can fly with {option['airline']} on {option['flight_date']} at {option['flight_time']}")
        print(f"From {flight['origin']} to {flight['destination']}")
        print()


For this price: 41.07
You can fly with Openskies for LEVEL France on 2023-08-08 at 09:55:00
You can fly with Openskies for LEVEL France on 2023-08-08 at 14:40:00
From MAD to LIS

For this price: 45.97
You can fly with TAP Portugal on 2023-08-08 at 13:35:00
You can fly with TAP Portugal on 2023-08-08 at 16:20:00
You can fly with TAP Portugal on 2023-08-08 at 17:55:00
From MAD to LIS

For this price: 33.97
You can fly with TAP Portugal on 2023-08-09 at 07:40:00
You can fly with TAP Portugal on 2023-08-09 at 11:05:00
You can fly with TAP Portugal on 2023-08-09 at 20:10:00
You can fly with TAP Portugal on 2023-08-09 at 22:15:00
From MAD to LIS

